In [2]:
import sqlite3

conn = sqlite3.connect('travel.sqlite')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [3]:
# Get all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("=== DATABASE TABLES ===")
for table in tables:
    print(f"\n{table[0]}")
    
    # Get column info for each table
    cursor.execute(f"PRAGMA table_info({table[0]})")
    columns = cursor.fetchall()
    
    for col in columns:
        print(f"  - {col[1]} ({col[2]})")
    
    # Get row count
    cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
    count = cursor.fetchone()[0]
    print(f"  Total rows: {count}")


=== DATABASE TABLES ===

aircrafts_data
  - aircraft_code (character(3))
  - model (jsonb)
  - range (INTEGER)
  Total rows: 9

airports_data
  - airport_code (character(3))
  - airport_name (jsonb)
  - city (jsonb)
  - coordinates (point)
  - timezone (TEXT)
  Total rows: 104

boarding_passes
  - ticket_no (character(13))
  - flight_id (INTEGER)
  - boarding_no (INTEGER)
  - seat_no (character varying(4))
  Total rows: 579686

bookings
  - book_ref (character(6))
  - book_date (timestamp with time zone)
  - total_amount (numeric(10,2))
  Total rows: 262788

flights
  - flight_id (INTEGER)
  - flight_no (character(6))
  - scheduled_departure (timestamp with time zone)
  - scheduled_arrival (timestamp with time zone)
  - departure_airport (character(3))
  - arrival_airport (character(3))
  - status (character varying(20))
  - aircraft_code (character(3))
  - actual_departure (timestamp with time zone)
  - actual_arrival (timestamp with time zone)
  Total rows: 33121

seats
  - aircraft_

Create a view to combine departure_airport with arrival_airport to form 'route'

In [6]:
# create view for flight routes
cursor.execute("DROP VIEW IF EXISTS flight_routes;")
cursor.execute("""
    CREATE VIEW flight_routes AS
    SELECT 
        f.*,
        f.departure_airport || ' → ' || f.arrival_airport AS route
    FROM flights f;""")

# Example: query the new view
cursor.execute("SELECT route, COUNT(*) as total FROM flight_routes GROUP BY route LIMIT 5;")
# cursor.execute("SELECT DISTINCT route, COUNT(*) as total FROM flight_routes GROUP BY route;")
flight_routes = [dict(row) for row in cursor.fetchall()]
print("\n=== Sample flight_routes data ===")
for row in flight_routes:
    print(row)


=== Sample flight_routes data ===
{'route': 'AAQ → EGO', 'total': 61}
{'route': 'AAQ → NOZ', 'total': 9}
{'route': 'AAQ → SVO', 'total': 61}
{'route': 'ABA → ARH', 'total': 8}
{'route': 'ABA → DME', 'total': 18}


1. Top 5 Least Punctual Routes

In [6]:
cursor.execute("""
    SELECT 
        route,
        -- Strip the '+HH' or '+HH:MM' timezone offset before passing to JULIANDAY
        ROUND(AVG(
            (
                JULIANDAY(SUBSTR(actual_arrival, 1, 19)) - 
                JULIANDAY(SUBSTR(scheduled_arrival, 1, 19))
            ) * 24 * 60
        ), 2) AS avg_delay_mins
    FROM flight_routes
    WHERE status = 'Arrived'
        AND actual_arrival IS NOT NULL
        AND scheduled_arrival IS NOT NULL
        -- Use the stripped timestamp for comparison to ensure valid JULIANDAY call
        AND JULIANDAY(SUBSTR(actual_arrival, 1, 19)) > JULIANDAY(SUBSTR(scheduled_arrival, 1, 19))
    GROUP BY route
    HAVING avg_delay_mins > 0
    ORDER BY avg_delay_mins DESC
    LIMIT 5;
""")

least_punctual_routes = [dict(row) for row in cursor.fetchall()]
print("\n=== Top 5 Least Punctual Routes ===")
for route in least_punctual_routes:
    print(route)

cursor.execute("""
    SELECT COUNT(*) FROM flight_routes
    WHERE actual_arrival IS NULL OR scheduled_arrival IS NULL;
""")
print("Flights missing timestamps:", cursor.fetchone()[0])



=== Top 5 Least Punctual Routes ===
{'route': 'VKO → DYR', 'avg_delay_mins': 143.0}
{'route': 'NOZ → AAQ', 'avg_delay_mins': 99.5}
{'route': 'CNN → LED', 'avg_delay_mins': 98.75}
{'route': 'KLF → OVB', 'avg_delay_mins': 91.8}
{'route': 'SGC → URS', 'avg_delay_mins': 73.83}
Flights missing timestamps: 0


2. Top 10 Routes by Average Occupancy Rate
    
This query calculates the average occupancy for each unique route combination and ranks them.

In [16]:
cursor.execute("""
    WITH FlightCapacity AS (
        -- 1. Determine the total seat capacity for each individual flight
        SELECT
            fr.flight_id,
            fr.route,
            COUNT(s.seat_no) AS total_seats
        FROM
            flight_routes AS fr
        JOIN
            seats AS s ON fr.aircraft_code = s.aircraft_code
        GROUP BY
            fr.flight_id, fr.route
    ),
    FlightBookings AS (
        -- 2. Count the actual number of occupied seats (bookings) for each flight
        SELECT
            flight_id,
            COUNT(ticket_no) AS booked_seats
        FROM
            boarding_passes
        GROUP BY
            flight_id
    )
    SELECT
        fc.route,
        -- Calculate the average of the individual flight occupancy percentages for this route
        ROUND(AVG(
            (fb.booked_seats * 100.0 / fc.total_seats)
        ), 2) AS avg_occupancy_percent
    FROM
        FlightCapacity AS fc
    JOIN
        FlightBookings AS fb ON fc.flight_id = fb.flight_id
    WHERE
        fc.total_seats > 0 -- Exclude flights with no capacity data
    GROUP BY
        fc.route
    ORDER BY
        avg_occupancy_percent DESC
    LIMIT 10;
""")

top_occupancy_routes = [dict(row) for row in cursor.fetchall()]
print("\n=== Top 10 Routes by Average Occupancy Rate ===")
for route in top_occupancy_routes:
    print(route)


=== Top 10 Routes by Average Occupancy Rate ===
{'route': 'NOZ → KJA', 'avg_occupancy_percent': 97.92}
{'route': 'CEE → LED', 'avg_occupancy_percent': 95.83}
{'route': 'EGO → AAQ', 'avg_occupancy_percent': 94.98}
{'route': 'SVX → SCW', 'avg_occupancy_percent': 93.77}
{'route': 'SVO → NNM', 'avg_occupancy_percent': 93.23}
{'route': 'SVO → KVX', 'avg_occupancy_percent': 92.22}
{'route': 'BZK → DME', 'avg_occupancy_percent': 92.12}
{'route': 'AER → KUF', 'avg_occupancy_percent': 90.72}
{'route': 'ESL → KGD', 'avg_occupancy_percent': 90.5}
{'route': 'SVO → ROV', 'avg_occupancy_percent': 90.27}


3. Busiest Routes by Total Passenger Volume (Market Share %)

This query calculates the total percentage of all tickets sold that belong to a particular route. This indicates the market share or demand concentration.
    $$\text{Market Share \%} = \text{ROUND}\left(\frac{\text{Route Tickets}}{\text{Grand Total}} \times 100.0, 2\right)$$

In [5]:
cursor.execute("""
    WITH RouteBookings AS (
        -- Calculate tickets sold per route
        SELECT
            fr.route,
            COUNT(tf.ticket_no) AS total_tickets_sold
        FROM
            flight_routes AS fr
        JOIN
            ticket_flights AS tf ON fr.flight_id = tf.flight_id
        GROUP BY
            fr.route
    ),
    TotalTickets AS (
        -- Calculate total tickets sold across ALL routes
        SELECT
            CAST(COUNT(ticket_no) AS REAL) AS grand_total
        FROM
            ticket_flights
    )
    SELECT
        rb.route,
        rb.total_tickets_sold,
        -- Calculate percentage of total market share
        ROUND((rb.total_tickets_sold * 100.0 / tt.grand_total), 2) AS market_share_percent
    FROM
        RouteBookings AS rb
    CROSS JOIN
        TotalTickets AS tt -- CROSS JOIN to apply the grand total to every route
    ORDER BY
        market_share_percent DESC
    LIMIT 10;
""")

busiest_routes = [dict(row) for row in cursor.fetchall()]

print("\n=== Top 10 Busiest Routes by Market Share % ===")
for route in busiest_routes:
    print(route)



=== Top 10 Busiest Routes by Market Share % ===
{'route': 'SVO → LED', 'total_tickets_sold': 16461, 'market_share_percent': 1.57}
{'route': 'DME → OVB', 'total_tickets_sold': 15903, 'market_share_percent': 1.52}
{'route': 'SVO → SVX', 'total_tickets_sold': 15879, 'market_share_percent': 1.52}
{'route': 'SVX → SVO', 'total_tickets_sold': 15433, 'market_share_percent': 1.48}
{'route': 'LED → SVO', 'total_tickets_sold': 15424, 'market_share_percent': 1.47}
{'route': 'OVB → DME', 'total_tickets_sold': 15402, 'market_share_percent': 1.47}
{'route': 'SVO → AER', 'total_tickets_sold': 15155, 'market_share_percent': 1.45}
{'route': 'AER → SVO', 'total_tickets_sold': 14661, 'market_share_percent': 1.4}
{'route': 'SVO → OVB', 'total_tickets_sold': 13160, 'market_share_percent': 1.26}
{'route': 'VKO → PEE', 'total_tickets_sold': 13070, 'market_share_percent': 1.25}


4. 10 Least Busy Routes by Total Passenger Volume (Market Share %)

In [14]:
cursor.execute("""
    WITH RouteBookings AS (
        -- Calculate tickets sold per route
        SELECT
            fr.route,
            COUNT(tf.ticket_no) AS total_tickets_sold
        FROM
            flight_routes AS fr
        JOIN
            ticket_flights AS tf ON fr.flight_id = tf.flight_id
        GROUP BY
            fr.route
    ),
    TotalTickets AS (
        -- Calculate total tickets sold across ALL routes
        SELECT
            CAST(COUNT(ticket_no) AS REAL) AS grand_total
        FROM
            ticket_flights
    )
    SELECT
        rb.route,
        rb.total_tickets_sold,
        -- Calculate percentage of total market share
        ROUND((rb.total_tickets_sold * 100.0 / tt.grand_total), 2) AS market_share_percent
    FROM
        RouteBookings AS rb
    CROSS JOIN
        TotalTickets AS tt
    -- Filter out routes that have no tickets sold to focus on active, low-volume routes
    WHERE
        rb.total_tickets_sold > 0
    ORDER BY
        market_share_percent ASC -- KEY CHANGE: Ascending order for LEAST busy
    LIMIT 10;
""")

least_busy_routes = [dict(row) for row in cursor.fetchall()]
print("\n=== 10 Least Busy Routes by Market Share % ===")
for route in least_busy_routes:
    print(route)


=== 10 Least Busy Routes by Market Share % ===
{'route': 'CEK → SWT', 'total_tickets_sold': 22, 'market_share_percent': 0.0}
{'route': 'LED → OVS', 'total_tickets_sold': 46, 'market_share_percent': 0.0}
{'route': 'MJZ → SVX', 'total_tickets_sold': 4, 'market_share_percent': 0.0}
{'route': 'OVS → LED', 'total_tickets_sold': 30, 'market_share_percent': 0.0}
{'route': 'ROV → NBC', 'total_tickets_sold': 30, 'market_share_percent': 0.0}
{'route': 'SVX → MJZ', 'total_tickets_sold': 7, 'market_share_percent': 0.0}
{'route': 'SWT → CEK', 'total_tickets_sold': 22, 'market_share_percent': 0.0}
{'route': 'UFA → OVS', 'total_tickets_sold': 43, 'market_share_percent': 0.0}
{'route': 'AER → KGP', 'total_tickets_sold': 109, 'market_share_percent': 0.01}
{'route': 'CEE → LED', 'total_tickets_sold': 79, 'market_share_percent': 0.01}


5. Top 10 Routes by Flight Volume (Number of Flights)

This query will be good for staffing analysis, identifying the routes with the highest operational frequency.

In [18]:
# Top 10 Routes by Flight Volume
cursor.execute("""
    SELECT
        fr.route,
        COUNT(fr.flight_id) AS total_flights
    FROM
        flight_routes AS fr
    GROUP BY
        fr.route
    ORDER BY
        total_flights DESC
    LIMIT 10;
""")

top_flight_volume_routes = [dict(row) for row in cursor.fetchall()]
print("\n=== Top 10 Routes by Flight Volume ===")
for route in top_flight_volume_routes:
    print(route)


=== Top 10 Routes by Flight Volume ===
{'route': 'SVO → LED', 'total_flights': 305}
{'route': 'LED → SVO', 'total_flights': 305}
{'route': 'LED → DME', 'total_flights': 244}
{'route': 'DME → LED', 'total_flights': 244}
{'route': 'VKO → LED', 'total_flights': 183}
{'route': 'VKO → BZK', 'total_flights': 183}
{'route': 'SVO → BZK', 'total_flights': 183}
{'route': 'LED → VKO', 'total_flights': 183}
{'route': 'DME → BZK', 'total_flights': 183}
{'route': 'BZK → VKO', 'total_flights': 183}


5. Top 3 Most & Least Profitable Routes

In [ ]:
#  Top 3 MOST profitable routes
cursor.execute("""
    SELECT
        fr.route,
        -- Sum of all ticket amounts sold for this route
        SUM(tf.amount) AS total_revenue
    FROM
        flight_routes AS fr
    JOIN
        ticket_flights AS tf ON fr.flight_id = tf.flight_id
    GROUP BY
        fr.route
    ORDER BY
        total_revenue DESC
    LIMIT 3;
""")

top_revenue_routes = [dict(row) for row in cursor.fetchall()]
print("\n=== Top 3 Routes by Revenue ===")
for route in top_revenue_routes:
    print(route)


=== Top 3 Routes by Revenue ===
{'route': 'DME → KHV', 'total_revenue': 753478300}
{'route': 'KHV → DME', 'total_revenue': 733797800}
{'route': 'DME → OVB', 'total_revenue': 548218900}


In [ ]:
# Top 3 LEAST profitable routes
cursor.execute("""
    SELECT
        fr.route,
        SUM(tf.amount) AS total_revenue
    FROM
        flight_routes AS fr
    JOIN
        ticket_flights AS tf ON fr.flight_id = tf.flight_id
    GROUP BY
        fr.route
    -- Filter for routes that have generated some revenue
    HAVING
        total_revenue > 0
    ORDER BY
        total_revenue ASC
    LIMIT 3;
""")

least_revenue_routes = [dict(row) for row in cursor.fetchall()]
print("\n=== Top 3 Least Profitable Routes ===")
for route in least_revenue_routes:
    print(route)


=== Top 3 Least Profitable Routes ===
{'route': 'MJZ → SVX', 'total_revenue': 118800}
{'route': 'CEK → SWT', 'total_revenue': 246400}
{'route': 'SWT → CEK', 'total_revenue': 246400}


6. Revenue by Fare Class & Route

In [45]:
cursor.execute("""
    SELECT
        fr.route,
        tf.fare_conditions,
        -- Sum of all ticket amounts sold for this route and fare class
        SUM(tf.amount) AS total_revenue_by_class
    FROM
        flight_routes AS fr
    JOIN
        ticket_flights AS tf ON fr.flight_id = tf.flight_id
    GROUP BY
        fr.route, tf.fare_conditions
    ORDER BY
        fr.route ASC, total_revenue_by_class DESC;
""")

revenue_by_class = [dict(row) for row in cursor.fetchall()]
print("\n=== Revenue by Fare Class & Route ===")
for entry in revenue_by_class:
    print(entry)


=== Revenue by Fare Class & Route ===
{'route': 'AAQ → EGO', 'fare_conditions': 'Economy', 'total_revenue_by_class': 23524800}
{'route': 'AAQ → EGO', 'fare_conditions': 'Business', 'total_revenue_by_class': 10092600}
{'route': 'AAQ → SVO', 'fare_conditions': 'Economy', 'total_revenue_by_class': 58351400}
{'route': 'AAQ → SVO', 'fare_conditions': 'Business', 'total_revenue_by_class': 18117000}
{'route': 'ABA → DME', 'fare_conditions': 'Economy', 'total_revenue_by_class': 27763400}
{'route': 'ABA → DME', 'fare_conditions': 'Business', 'total_revenue_by_class': 15857000}
{'route': 'ABA → OVB', 'fare_conditions': 'Economy', 'total_revenue_by_class': 2534600}
{'route': 'ABA → TOF', 'fare_conditions': 'Economy', 'total_revenue_by_class': 2391200}
{'route': 'AER → EGO', 'fare_conditions': 'Economy', 'total_revenue_by_class': 1621200}
{'route': 'AER → GOJ', 'fare_conditions': 'Economy', 'total_revenue_by_class': 3006600}
{'route': 'AER → KGP', 'fare_conditions': 'Economy', 'total_revenue_by_c

7. Aircraft Type by Route

In [7]:
cursor.execute("""
    SELECT
        fr.route,
        fr.aircraft_code,
        ad.model AS aircraft_model, -- Assuming 'model' is a readable TEXT field or simple JSON
        COUNT(fr.flight_id) AS total_flights_on_route
    FROM
        flight_routes AS fr
    JOIN
        aircrafts_data AS ad ON fr.aircraft_code = ad.aircraft_code
    GROUP BY
        fr.route, fr.aircraft_code, ad.model
    ORDER BY
        fr.route ASC, total_flights_on_route DESC;
""")

aircrafts_per_route = [dict(row) for row in cursor.fetchall()]
print("\n=== Aircrafts Used per Route ===")
for entry in aircrafts_per_route:
    print(f"Route: {entry['route']}, Aircraft Code: {entry['aircraft_code']}, Model: {entry['aircraft_model']}, Total Flights: {entry['total_flights_on_route']}")


=== Aircrafts Used per Route ===
Route: AAQ → EGO, Aircraft Code: SU9, Model: {"en": "Sukhoi Superjet-100", "ru": "Сухой Суперджет-100"}, Total Flights: 61
Route: AAQ → NOZ, Aircraft Code: 733, Model: {"en": "Boeing 737-300", "ru": "Боинг 737-300"}, Total Flights: 9
Route: AAQ → SVO, Aircraft Code: 733, Model: {"en": "Boeing 737-300", "ru": "Боинг 737-300"}, Total Flights: 61
Route: ABA → ARH, Aircraft Code: 319, Model: {"en": "Airbus A319-100", "ru": "Аэробус A319-100"}, Total Flights: 8
Route: ABA → DME, Aircraft Code: 319, Model: {"en": "Airbus A319-100", "ru": "Аэробус A319-100"}, Total Flights: 18
Route: ABA → GRV, Aircraft Code: 733, Model: {"en": "Boeing 737-300", "ru": "Боинг 737-300"}, Total Flights: 9
Route: ABA → KYZ, Aircraft Code: CN1, Model: {"en": "Cessna 208 Caravan", "ru": "Сессна 208 Караван"}, Total Flights: 17
Route: ABA → OVB, Aircraft Code: CN1, Model: {"en": "Cessna 208 Caravan", "ru": "Сессна 208 Караван"}, Total Flights: 122
Route: ABA → TOF, Aircraft Code: CN

8. Top 3 Most Visited Destination

In [5]:
cursor.execute("""
    SELECT
        f.arrival_airport AS airport_code,
        ad.city AS destination_city, -- Assuming 'city' is a readable TEXT field
        COUNT(f.flight_id) AS total_arrivals
    FROM
        flights AS f
    JOIN
        airports_data AS ad ON f.arrival_airport = ad.airport_code
    GROUP BY
        f.arrival_airport, destination_city
    ORDER BY
        total_arrivals DESC
    LIMIT 3;
""")

most_visited_destinations = [dict(row) for row in cursor.fetchall()]
print("\n=== Top 3 Most Visited Destinations ===")
for entry in most_visited_destinations:
    print(f"Airport Code: {entry['airport_code']}, City: {entry['destination_city']}, Total Arrivals: {entry['total_arrivals']}")


=== Top 3 Most Visited Destinations ===
Airport Code: DME, City: {"en": "Moscow", "ru": "Москва"}, Total Arrivals: 3217
Airport Code: SVO, City: {"en": "Moscow", "ru": "Москва"}, Total Arrivals: 2982
Airport Code: LED, City: {"en": "St. Petersburg", "ru": "Санкт-Петербург"}, Total Arrivals: 1902


9. Top 10 planes with most mileage

In [ ]:
cursor.execute("""
    SELECT
        f.aircraft_code,
        ad.model AS aircraft_model,
        -- Sum the aircraft's range for every flight it completed
        SUM(ad.range) AS total_utilization_proxy_miles
    FROM
        flights AS f
    JOIN
        aircrafts_data AS ad ON f.aircraft_code = ad.aircraft_code
    WHERE
        f.status = 'Arrived' -- Only count flights that were completed
    GROUP BY
        f.aircraft_code, aircraft_model
    ORDER BY
        total_utilization_proxy_miles DESC
    LIMIT 10;
""")

most_utilized_aircrafts = [dict(row) for row in cursor.fetchall()]
print("\n=== Top 10 Most Utilized Aircrafts by Proxy Miles ===")
for entry in most_utilized_aircrafts:
    print(f"Aircraft Code: {entry['aircraft_code']}, Model: {entry['aircraft_model']}, Total Proxy Miles: {entry['total_utilization_proxy_miles']}")


=== Top 10 Most Utilized Aircrafts by Proxy Miles ===
Aircraft Code: SU9, Model: {"en": "Sukhoi Superjet-100", "ru": "Сухой Суперджет-100"}, Total Proxy Miles: 12876000
Aircraft Code: CR2, Model: {"en": "Bombardier CRJ-200", "ru": "Бомбардье CRJ-200"}, Total Proxy Miles: 12339000
Aircraft Code: CN1, Model: {"en": "Cessna 208 Caravan", "ru": "Сессна 208 Караван"}, Total Proxy Miles: 5608800
Aircraft Code: 321, Model: {"en": "Airbus A321-200", "ru": "Аэробус A321-200"}, Total Proxy Miles: 5499200
Aircraft Code: 763, Model: {"en": "Boeing 767-300", "ru": "Боинг 767-300"}, Total Proxy Miles: 4834800
Aircraft Code: 319, Model: {"en": "Airbus A319-100", "ru": "Аэробус A319-100"}, Total Proxy Miles: 4180800
Aircraft Code: 773, Model: {"en": "Boeing 777-300", "ru": "Боинг 777-300"}, Total Proxy Miles: 3407700
Aircraft Code: 733, Model: {"en": "Boeing 737-300", "ru": "Боинг 737-300"}, Total Proxy Miles: 2713200


10. Routes Taken by Aircraft (SU9, plane with most mileage)

In [8]:
cursor.execute("""
    SELECT
        fr.flight_id,
        fr.route,
        SUBSTR(fr.scheduled_departure, 1, 16) AS scheduled_departure_time, -- Formatting for readability
        fr.status
    FROM
        flight_routes AS fr
    WHERE
        fr.aircraft_code = 'SU9' -- Targeting the most utilized aircraft
    ORDER BY
        fr.scheduled_departure ASC;
""")

routes_taken_by_su9 = [dict(row) for row in cursor.fetchall()]
print("\n=== Routes Taken by Aircraft SU9 ===")
for entry in routes_taken_by_su9:
    print(f"Flight ID: {entry['flight_id']}, Route: {entry['route']}, Scheduled Departure: {entry['scheduled_departure_time']}, Status: {entry['status']}")



=== Routes Taken by Aircraft SU9 ===
Flight ID: 26680, Route: VVO → KHV, Scheduled Departure: 2017-07-16 02:45, Status: Arrived
Flight ID: 25187, Route: BAX → YKS, Scheduled Departure: 2017-07-16 05:35, Status: Arrived
Flight ID: 27055, Route: TOF → SVO, Scheduled Departure: 2017-07-16 06:00, Status: Arrived
Flight ID: 26339, Route: KHV → BQS, Scheduled Departure: 2017-07-16 06:15, Status: Arrived
Flight ID: 20931, Route: NSK → SVX, Scheduled Departure: 2017-07-16 06:40, Status: Arrived
Flight ID: 15025, Route: PEE → ULV, Scheduled Departure: 2017-07-16 07:15, Status: Arrived
Flight ID: 21969, Route: HMA → DME, Scheduled Departure: 2017-07-16 07:15, Status: Arrived
Flight ID: 14158, Route: KJA → OVS, Scheduled Departure: 2017-07-16 07:25, Status: Arrived
Flight ID: 24836, Route: TJM → URJ, Scheduled Departure: 2017-07-16 07:30, Status: Arrived
Flight ID: 22527, Route: OVS → DME, Scheduled Departure: 2017-07-16 07:45, Status: Arrived
Flight ID: 27692, Route: NOZ → OVB, Scheduled Depart